In [3]:
import time
import os
from IPython.display import clear_output
'''
try:
  from google.colab import drive
  drive.mount('/content/drive/')
  %cd drive/My\ Drive/Project/project
  !git pull
except OSError:
  !kill -9 -1
'''
from engine import train_one_epoch, evaluate
import utils

try:
  from read_data import system_load
  from dataset import load_dataset
  from classes import class_list
except ModuleNotFoundError as e:
  print(e)
  !pip install geopandas
finally:
  from read_data import system_load
  from dataset import load_dataset
  from classes import class_list
  clear_output()

import torch
import torchvision

meta, root_dir = system_load(init=False)
datasets, dataset_lengths = load_dataset(meta, root_dir, include=[73, 18, 23, 19], workers=0)
classes = class_list()

meta loaded!
correspondence of class names and labels: {73: 0, 18: 1, 23: 2, 19: 3}


In [4]:
def avg_acc(sozluk):
    return np.average(np.array([i for i in sozluk.values()]))
    
def printm():
    try:
        import psutil
        import humanize
        import os
        import GPUtil as GPU
    except ModuleNotFoundError as e:
        print(e)
        !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
        !pip install gputil
        !pip install psutil
        !pip install humanize
    finally:
        import psutil
        import humanize
        import os
        import GPUtil as GPU
        
    clear_output()
    GPUs = GPU.getGPUs()
    # XXX: only one GPU on Colab and isn’t guaranteed
    gpu = GPUs[0]

    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))


In [5]:
def train_model(model, dataloaders, dataset_lengths, optimizer, scheduler, num_epochs, device="cpu"):
    print("start training. . .")
    since = time.time()
    
    #loss_values = {'loss_classifier': [], 'loss_box_reg': [], 'loss_objectness': [], 'loss_rpn_box_reg': []}
    loss_values = []
    #acc_values = {'acc_classifier': [], 'acc_box_reg': [], 'acc_objectness': [], 'acc_rpn_box_reg': []}
    #best_model_wts = copy.deepcopy(model.state_dict())
    #best_acc = 0.0

    for epoch in range(num_epochs):
        # Each epoch has a training and validation phase
        for phase in ['train', 'validate']:
            train_one_epoch(model, optimizer, dataloaders["train"], device, num_epochs, print_freq=50)

    return model


In [5]:
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn

from torchvision.models.detection.rpn import AnchorGenerator


model = fasterrcnn_resnet50_fpn(pretrained=True)

''' Backbone model creation'''

backbone = torchvision.models.mobilenet_v2(pretrained=True).features
backbone.out_channels = 1280


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
# move model to the right device
model.to(device)
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.1)
# let's train it for 10 epochs
num_epochs = 10

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\Serkan/.cache\torch\checkpoints\mobilenet_v2-b0353104.pth
100.0%


In [ ]:
train_model(model, datasets, dataset_lengths, optimizer, lr_scheduler, num_epochs, device="cpu")

start training. . .
Epoch 0/9
----------


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


{'loss_classifier': tensor(5.0492, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(0.2911, grad_fn=<DivBackward0>), 'loss_objectness': tensor(3.1338, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(13.7410, grad_fn=<DivBackward0>)}
